In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torchinfo

# Python basic

## numpy.repeat

In [168]:
m= np.round(np.random.uniform(0,2,size=(1,1,2,3)), 2)
m2= np.repeat(np.repeat(m,3,axis=1),2,axis=0)
print(f'm= {m}, {m.shape}')
print(f'm2= {m2}, {m2.shape}')

m= [[[[1.61 0.68 0.06]
   [1.53 0.2  0.54]]]], (1, 1, 2, 3)
m2= [[[[1.61 0.68 0.06]
   [1.53 0.2  0.54]]

  [[1.61 0.68 0.06]
   [1.53 0.2  0.54]]

  [[1.61 0.68 0.06]
   [1.53 0.2  0.54]]]


 [[[1.61 0.68 0.06]
   [1.53 0.2  0.54]]

  [[1.61 0.68 0.06]
   [1.53 0.2  0.54]]

  [[1.61 0.68 0.06]
   [1.53 0.2  0.54]]]], (2, 3, 2, 3)


## Einsum

In [10]:
A= np.round(np.random.uniform(0,2,size=(3,4)), 2)
B= np.round(np.random.uniform(-1,1,size=(4,3)), 2)
print(f'A= {A}')
print(f'B= {B}')
print(f'A@B= {A@B}')
print(f"ik,kj->ij= {np.einsum('ik,kj->ij',A,B)}")
print(f'A*(B.T)= {A*(B.T)}')
print(f"ij,ji->ij= {np.einsum('ij,ji->ij',A,B)}")

A= [[0.39 0.21 0.17 1.15]
 [1.01 0.96 0.93 1.3 ]
 [0.53 1.79 1.8  1.5 ]]
B= [[-0.74  0.58 -0.18]
 [ 0.32 -0.65 -0.72]
 [-0.5  -0.86  0.09]
 [ 0.87  0.58  0.83]]
A@B= [[ 0.6941  0.6105  0.7484]
 [ 0.2258 -0.084   0.2897]
 [ 0.5856 -1.5341  0.0228]]
ik,kj->ij= [[ 0.6941  0.6105  0.7484]
 [ 0.2258 -0.084   0.2897]
 [ 0.5856 -1.5341  0.0228]]
A*(B.T)= [[-0.2886  0.0672 -0.085   1.0005]
 [ 0.5858 -0.624  -0.7998  0.754 ]
 [-0.0954 -1.2888  0.162   1.245 ]]
ij,ji->ij= [[-0.2886  0.0672 -0.085   1.0005]
 [ 0.5858 -0.624  -0.7998  0.754 ]
 [-0.0954 -1.2888  0.162   1.245 ]]


## Container class

In [9]:
#Local variable holder.
class TLocalVar:
  def __init__(self, l):
    self.__dict__= l

In [13]:
def test_f(a):
  b= 10
  print(locals())
  l= TLocalVar(locals())
  print(l)
  # print(l['a'])
  print(l.a,l.b)
  l.a*= 100
  l.b*= 100
  print(l.a,l.b)
  print(a,b)
  locals()['a']*= 100
  print(a,b)
test_f(100)

{'b': 10, 'a': 100}
100 10
10000 1000
100 10
100 10


In [14]:
#Container class to share variables.
class TContainer(object):
  def __init__(self):
    pass
  def __del__(self):
    pass
  def __str__(self):
    return str(self.__dict__)
  def __repr__(self):
    return str(self.__dict__)
  def __iter__(self):
    return self.__dict__.itervalues()
  def items(self):
    return self.__dict__.items()
  def iteritems(self):
    return self.__dict__.iteritems()
  def keys(self):
    return self.__dict__.keys()
  def values(self):
    return self.__dict__.values()
  def __getitem__(self,key):
    return self.__dict__[key]
  def __setitem__(self,key,value):
    self.__dict__[key]= value
  def __delitem__(self,key):
    del self.__dict__[key]
  def __contains__(self,key):
    return key in self.__dict__
  def Cleanup(self):
    keys= self.__dict__.keys()
    for k in keys:
      self.__dict__[k]= None
      del self.__dict__[k]

In [15]:
l= TContainer()

In [16]:
for l.i in range(3):
  print(l.i)

0
1
2


## Discretization

In [30]:
class TDiscretizer(object):
  def __init__(self, xmin, xmax, n_div):
    self.xmin, self.xmax, self.n_div= xmin, xmax, n_div
    self.dx= (xmax-xmin)/n_div
    self.ptcls_max= torch.tensor(self.n_div-1)
    self.ptcls_min= torch.tensor(0)
  def NumClasses(self):  return self.n_div
  def Encode(self, x):  
    if isinstance(x,torch.Tensor):
      return torch.minimum(self.ptcls_max,torch.maximum(self.ptcls_min,torch.floor((x-self.xmin)/self.dx))).long()
    return np.minimum(self.n_div-1,np.maximum(0,np.floor((x-self.xmin)/self.dx)))
  def Decode(self, c):
    if isinstance(c, torch.Tensor):
      return (self.xmin+0.5*self.dx)+self.dx*torch.minimum(self.ptcls_max,torch.maximum(self.ptcls_min,c))
    return (self.xmin+0.5*self.dx)+self.dx*np.minimum(self.n_div-1,np.maximum(0,c))
disc= TDiscretizer(-2.5, 3.5, 12)
print(f'disc.NumClasses()= {disc.NumClasses()}')
# np.minimum(3,np.array(range(10))), np.minimum(3,0)
# torch.minimum(torch.tensor(3),torch.tensor(range(10))), torch.minimum(3,0)
# torch.tensor(range(10))-4
X= [x for x in np.linspace(-3.,4.,10)]
print(f'X= {X}')
print(f'disc.Encode(x)= {[disc.Encode(x) for x in X]}')
print(f'disc.Encode(np.array(X))= {disc.Encode(np.array(X))}')
print(f'disc.Encode(torch.tensor(X).float())= {disc.Encode(torch.tensor(X).float())}')
C= np.random.randint(-1,15, size=(10))
print(f'C= {C}')
print(f'disc.Decode(c)= {[disc.Decode(c) for c in C]}')
print(f'disc.Decode(np.array(C))= {disc.Decode(np.array(C))}')
print(f'disc.Decode(torch.tensor(C))= {disc.Decode(torch.tensor(C))}')

disc.NumClasses()= 12
X= [-3.0, -2.2222222222222223, -1.4444444444444444, -0.6666666666666665, 0.11111111111111116, 0.8888888888888888, 1.666666666666667, 2.4444444444444446, 3.2222222222222223, 4.0]
disc.Encode(x)= [0.0, 0.0, 2.0, 3.0, 5.0, 6.0, 8.0, 9.0, 11.0, 11.0]
disc.Encode(np.array(X))= [ 0.  0.  2.  3.  5.  6.  8.  9. 11. 11.]
disc.Encode(torch.tensor(X).float())= tensor([ 0,  0,  2,  3,  5,  6,  8,  9, 11, 11])
C= [10  9 -1 11 10  6  4  6  0 -1]
disc.Decode(c)= [2.75, 2.25, -2.25, 3.25, 2.75, 0.75, -0.25, 0.75, -2.25, -2.25]
disc.Decode(np.array(C))= [ 2.75  2.25 -2.25  3.25  2.75  0.75 -0.25  0.75 -2.25 -2.25]
disc.Decode(torch.tensor(C))= tensor([ 2.7500,  2.2500, -2.2500,  3.2500,  2.7500,  0.7500, -0.2500,  0.7500,
        -2.2500, -2.2500])


# Torch Tensor

## shape and view

In [2]:
# n_batch= 1
n_batch= 4

In [3]:
x= torch.tensor(np.zeros((n_batch,3,32,32))).float()

In [4]:
x.shape

torch.Size([4, 3, 32, 32])

In [5]:
x[0].numel()

3072

In [6]:
x.view(x.size(0), -1).shape

torch.Size([4, 3072])

In [7]:
l=torch.nn.Linear(x[0].numel(), 10)
l

Linear(in_features=3072, out_features=10, bias=True)

In [8]:
l(x[0].view(-1))

tensor([ 0.0170, -0.0096, -0.0093, -0.0021,  0.0015,  0.0124,  0.0121,  0.0003,
        -0.0003, -0.0002], grad_fn=<AddBackward0>)

In [9]:
l(x.view(x.size(0), -1))

tensor([[ 0.0170, -0.0096, -0.0093, -0.0021,  0.0015,  0.0124,  0.0121,  0.0003,
         -0.0003, -0.0002],
        [ 0.0170, -0.0096, -0.0093, -0.0021,  0.0015,  0.0124,  0.0121,  0.0003,
         -0.0003, -0.0002],
        [ 0.0170, -0.0096, -0.0093, -0.0021,  0.0015,  0.0124,  0.0121,  0.0003,
         -0.0003, -0.0002],
        [ 0.0170, -0.0096, -0.0093, -0.0021,  0.0015,  0.0124,  0.0121,  0.0003,
         -0.0003, -0.0002]], grad_fn=<AddmmBackward>)

## cat (concatenate) and stack

In [10]:
x1= torch.from_numpy(np.array([2,3,4])).float()
x2= torch.from_numpy(np.array([10,22,12])).float()
print(f'x1={x1}')
print(f'x2={x2}')
print(f'torch.cat={torch.cat((x1,x2),axis=0)}')
print(f'torch.stack={torch.stack((x1,x2),axis=0)}')

x1=tensor([2., 3., 4.])
x2=tensor([10., 22., 12.])
torch.cat=tensor([ 2.,  3.,  4., 10., 22., 12.])
torch.stack=tensor([[ 2.,  3.,  4.],
        [10., 22., 12.]])


## view and flatten

In [113]:
n_batch= 64
x= torch.tensor(np.zeros((n_batch,128,6,6))).float()

In [114]:
x.shape

torch.Size([64, 128, 6, 6])

In [115]:
x.view(x.shape[0], -1).shape

torch.Size([64, 4608])

In [116]:
flatten= torch.nn.Flatten()
flatten(x).shape

torch.Size([64, 4608])

In [117]:
torch.flatten(x,1).shape

torch.Size([64, 4608])

## unflatten

In [122]:
torch.nn.Unflatten(1,(128,6,6))(torch.flatten(x,1)).shape

torch.Size([64, 128, 6, 6])

## Unsqueeze

In [89]:
x= torch.zeros(3,10,10)
x.shape, x.unsqueeze(0).shape

(torch.Size([3, 10, 10]), torch.Size([1, 3, 10, 10]))

# Torch Module

## Sequential

In [10]:
from ay_torch import Noop, TNoop
# print(isinstance(TNoop(),torch.nn.Module))
x= torch.ones(3)
l1= torch.nn.Sequential(torch.nn.Linear(3,3),torch.nn.Linear(3,4))
l2= torch.nn.Sequential(torch.nn.Linear(3,3),None,torch.nn.Linear(3,4))
l2b= torch.nn.Sequential(torch.nn.Linear(3,3),TNoop(),torch.nn.Linear(3,4))
l3= torch.nn.Sequential(*filter(lambda x:x is not None, (torch.nn.Linear(3,3),None,torch.nn.Linear(3,4))))
print(f'l1={l1}')
print(f'l2={l2}')
print(f'l2b={l2b}')
print(f'l3={l3}')
print(f'x={x}')
print(f'l1(x)={l1(x)}')
# print(f'l2(x)={l2(x)}')  #ERROR: TypeError: 'NoneType' object is not callable
print(f'l2b(x)={l2b(x)}')
print(f'l3(x)={l3(x)}')

l1=Sequential(
  (0): Linear(in_features=3, out_features=3, bias=True)
  (1): Linear(in_features=3, out_features=4, bias=True)
)
l2=Sequential(
  (0): Linear(in_features=3, out_features=3, bias=True)
  (1): None
  (2): Linear(in_features=3, out_features=4, bias=True)
)
l2b=Sequential(
  (0): Linear(in_features=3, out_features=3, bias=True)
  (1): TNoop()
  (2): Linear(in_features=3, out_features=4, bias=True)
)
l3=Sequential(
  (0): Linear(in_features=3, out_features=3, bias=True)
  (1): Linear(in_features=3, out_features=4, bias=True)
)
x=tensor([1., 1., 1.])
l1(x)=tensor([-0.2396,  0.3478, -0.1649, -0.6443], grad_fn=<AddBackward0>)
l2b(x)=tensor([ 0.1232, -0.1427, -0.7143,  0.4091], grad_fn=<AddBackward0>)
l3(x)=tensor([-0.1548, -0.1635,  0.3904, -0.3709], grad_fn=<AddBackward0>)


In [44]:
# l3.append(torch.nn.Linear(4,3))  #AttributeError: 'Sequential' object has no attribute 'append'
l4= torch.nn.Sequential(l3, torch.nn.Linear(4,3))
print(f'l4={l4}')
print(f'l4(x)={l4(x)}')

l4=Sequential(
  (0): Sequential(
    (0): Linear(in_features=3, out_features=3, bias=True)
    (1): Linear(in_features=3, out_features=4, bias=True)
  )
  (1): Linear(in_features=4, out_features=3, bias=True)
)
l4(x)=tensor([0.4532, 0.0684, 0.5474], grad_fn=<AddBackward0>)


## BatchNormal

In [74]:
x= torch.tensor(range(20))/2.0
x= x[torch.randperm(x.shape[0])].reshape(-1,5)
print(f'x= {x} (shape={x.shape})')
bn= torch.nn.BatchNorm1d(5)
# bn.bias.data.fill_(1e-3)
bn.weight.data.fill_(0.)
# bn.bias.data= torch.Tensor([1.,2.,3.,4.,5.])*0.1
# bn.weight.data= torch.Tensor([5.,4.,3.,2.,1.])*0.1
bn.weight.data.fill_(1.)
print(f'bn(x)= {bn(x)}')

x= tensor([[7.5000, 8.0000, 3.5000, 2.0000, 8.5000],
        [9.0000, 5.0000, 4.0000, 6.5000, 1.0000],
        [9.5000, 5.5000, 3.0000, 1.5000, 0.5000],
        [7.0000, 0.0000, 6.0000, 4.5000, 2.5000]]) (shape=torch.Size([4, 5]))
bn(x)= tensor([[-0.7276,  1.1630, -0.5488, -0.8078,  1.6853],
        [ 0.7276,  0.1292, -0.1098,  1.4291, -0.6663],
        [ 1.2127,  0.3015, -0.9879, -1.0563, -0.8231],
        [-1.2127, -1.5937,  1.6465,  0.4350, -0.1960]],
       grad_fn=<NativeBatchNormBackward>)


In [75]:
bn.weight.data, bn.bias.data

(tensor([1., 1., 1., 1., 1.]), tensor([0., 0., 0., 0., 0.]))

In [77]:
torch.mean(x,axis=0), torch.var(x,axis=0)
(x-torch.mean(x,axis=0))/torch.sqrt(torch.var(x,axis=0)) * bn.weight + bn.bias

tensor([[-0.6301,  1.0072, -0.4753, -0.6996,  1.4595],
        [ 0.6301,  0.1119, -0.0951,  1.2377, -0.5770],
        [ 1.0502,  0.2611, -0.8555, -0.9148, -0.7128],
        [-1.0502, -1.3802,  1.4259,  0.3767, -0.1697]], grad_fn=<AddBackward0>)

In [82]:
inn= torch.nn.InstanceNorm1d(5, affine=True)
# inn.bias.data.fill_(1e-3)
# inn.weight.data.fill_(0.)
# inn.bias.data= torch.Tensor([1.,2.,3.,4.,5.])*0.1
# inn.weight.data= torch.Tensor([5.,4.,3.,2.,1.])*0.1
# inn.weight.data.fill_(1.)
print(f'inn(x)= {inn(x)}')

ValueError: InstanceNorm1d returns 0-filled tensor to 2D tensor.This is because InstanceNorm1d reshapes inputs to(1, N * C, ...) from (N, C,...) and this makesvariances 0.

## Max module

In [91]:
x= torch.tensor(range(150))/2.0
x= x[torch.randperm(x.shape[0])].reshape(-1,3,5,5)
x

tensor([[[[59.0000, 17.0000, 55.5000, 56.5000, 58.5000],
          [24.0000, 39.5000, 50.5000, 62.0000, 72.5000],
          [28.0000, 42.5000, 41.0000, 51.0000, 36.5000],
          [54.0000, 70.0000, 35.5000, 20.0000, 28.5000],
          [26.0000, 16.0000,  6.0000, 70.5000, 49.0000]],

         [[19.0000,  0.0000, 23.0000, 48.5000, 14.0000],
          [65.5000, 36.0000,  4.0000, 64.0000, 12.0000],
          [25.0000, 34.0000, 35.0000,  2.5000, 30.5000],
          [61.0000, 54.5000, 25.5000, 60.0000, 69.0000],
          [65.0000,  1.5000, 68.0000, 45.0000, 32.0000]],

         [[19.5000, 31.0000, 57.5000, 13.0000, 46.0000],
          [55.0000, 14.5000, 53.5000,  2.0000, 43.5000],
          [49.5000, 67.5000, 46.5000, 15.5000, 61.5000],
          [34.5000,  3.0000, 10.5000, 44.0000,  1.0000],
          [ 9.0000, 38.5000, 60.5000, 33.5000, 72.0000]]],


        [[[33.0000, 73.5000, 41.5000,  4.5000, 18.5000],
          [31.5000,  3.5000,  5.0000, 63.0000, 62.5000],
          [69.5000, 23.

In [99]:
torch.max(x, axis=1)[0].unsqueeze(1).shape

torch.Size([2, 1, 5, 5])

In [103]:
class TMax(torch.nn.Module):
  def forward(self, x):
    return torch.max(x, axis=1)[0].unsqueeze(1)

In [104]:
m= TMax()
m(x)

tensor([[[[59.0000, 31.0000, 57.5000, 56.5000, 58.5000],
          [65.5000, 39.5000, 53.5000, 64.0000, 72.5000],
          [49.5000, 67.5000, 46.5000, 51.0000, 61.5000],
          [61.0000, 70.0000, 35.5000, 60.0000, 69.0000],
          [65.0000, 38.5000, 68.0000, 70.5000, 72.0000]]],


        [[[68.5000, 73.5000, 67.0000, 26.5000, 66.0000],
          [31.5000, 51.5000, 57.0000, 66.5000, 62.5000],
          [69.5000, 58.0000, 71.0000, 71.5000, 24.5000],
          [16.5000, 64.5000, 56.0000, 48.0000, 47.5000],
          [40.0000, 74.0000, 74.5000, 53.0000, 52.5000]]]])

## AvgPool2d and Upsample

In [124]:
img= torch.from_numpy(np.random.uniform(0,1,size=(1,1,8,8))).float()
print(img.shape)
img

torch.Size([1, 1, 8, 8])


tensor([[[[0.2388, 0.1915, 0.6603, 0.9222, 0.2945, 0.0357, 0.7832, 0.1920],
          [0.4177, 0.8654, 0.8985, 0.7739, 0.7018, 0.4917, 0.0838, 0.7546],
          [0.2234, 0.6977, 0.2054, 0.9994, 0.8951, 0.7883, 0.3776, 0.6775],
          [0.4321, 0.6387, 0.0568, 0.0692, 0.7113, 0.3858, 0.5290, 0.1512],
          [0.7873, 0.7903, 0.5976, 0.1046, 0.5114, 0.9380, 0.3291, 0.2403],
          [0.2715, 0.7936, 0.8609, 0.6254, 0.4417, 0.2772, 0.4428, 0.8376],
          [0.5523, 0.5319, 0.9425, 0.3161, 0.6561, 0.8714, 0.4156, 0.9550],
          [0.4729, 0.7059, 0.4009, 0.0883, 0.2010, 0.8251, 0.3593, 0.3616]]]])

In [125]:
img2= torch.nn.AvgPool2d(kernel_size=2, stride=None, padding=0, ceil_mode=True)(img)
print(img2.shape)
img2

torch.Size([1, 1, 4, 4])


tensor([[[[0.4283, 0.8137, 0.3809, 0.4534],
          [0.4980, 0.3327, 0.6951, 0.4338],
          [0.6607, 0.5471, 0.5421, 0.4624],
          [0.5658, 0.4369, 0.6384, 0.5228]]]])

In [126]:
img3= torch.nn.Upsample(scale_factor=2, mode='nearest', align_corners=None)(img2)
print(img3.shape)
img3

torch.Size([1, 1, 8, 8])


tensor([[[[0.4283, 0.4283, 0.8137, 0.8137, 0.3809, 0.3809, 0.4534, 0.4534],
          [0.4283, 0.4283, 0.8137, 0.8137, 0.3809, 0.3809, 0.4534, 0.4534],
          [0.4980, 0.4980, 0.3327, 0.3327, 0.6951, 0.6951, 0.4338, 0.4338],
          [0.4980, 0.4980, 0.3327, 0.3327, 0.6951, 0.6951, 0.4338, 0.4338],
          [0.6607, 0.6607, 0.5471, 0.5471, 0.5421, 0.5421, 0.4624, 0.4624],
          [0.6607, 0.6607, 0.5471, 0.5471, 0.5421, 0.5421, 0.4624, 0.4624],
          [0.5658, 0.5658, 0.4369, 0.4369, 0.6384, 0.6384, 0.5228, 0.5228],
          [0.5658, 0.5658, 0.4369, 0.4369, 0.6384, 0.6384, 0.5228, 0.5228]]]])

In [131]:
img4= torch.nn.Upsample(size=(10,10), mode='nearest', align_corners=None)(img2)
print(img4.shape)
img4

torch.Size([1, 1, 10, 10])


tensor([[[[0.4283, 0.4283, 0.4283, 0.8137, 0.8137, 0.3809, 0.3809, 0.3809,
           0.4534, 0.4534],
          [0.4283, 0.4283, 0.4283, 0.8137, 0.8137, 0.3809, 0.3809, 0.3809,
           0.4534, 0.4534],
          [0.4283, 0.4283, 0.4283, 0.8137, 0.8137, 0.3809, 0.3809, 0.3809,
           0.4534, 0.4534],
          [0.4980, 0.4980, 0.4980, 0.3327, 0.3327, 0.6951, 0.6951, 0.6951,
           0.4338, 0.4338],
          [0.4980, 0.4980, 0.4980, 0.3327, 0.3327, 0.6951, 0.6951, 0.6951,
           0.4338, 0.4338],
          [0.6607, 0.6607, 0.6607, 0.5471, 0.5471, 0.5421, 0.5421, 0.5421,
           0.4624, 0.4624],
          [0.6607, 0.6607, 0.6607, 0.5471, 0.5471, 0.5421, 0.5421, 0.5421,
           0.4624, 0.4624],
          [0.6607, 0.6607, 0.6607, 0.5471, 0.5471, 0.5421, 0.5421, 0.5421,
           0.4624, 0.4624],
          [0.5658, 0.5658, 0.5658, 0.4369, 0.4369, 0.6384, 0.6384, 0.6384,
           0.5228, 0.5228],
          [0.5658, 0.5658, 0.5658, 0.4369, 0.4369, 0.6384, 0.6384, 0.6384

## Unfold+bmm and Conv2d

In [16]:
# img= torch.from_numpy(np.random.uniform(0,1,size=(1,1,4,4))).float()
img= torch.tensor(range(16)).float().reshape(1,1,4,4)
print(img.shape)
img

torch.Size([1, 1, 4, 4])


tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [21]:
unfold= torch.nn.Unfold(kernel_size=(2,2))
print(f'parameters: {[p for p in unfold.parameters()]}')
img_u= unfold(img)
print(img_u.shape)
img_u

parameters: []
torch.Size([1, 4, 9])


tensor([[[ 0.,  1.,  2.,  4.,  5.,  6.,  8.,  9., 10.],
         [ 1.,  2.,  3.,  5.,  6.,  7.,  9., 10., 11.],
         [ 4.,  5.,  6.,  8.,  9., 10., 12., 13., 14.],
         [ 5.,  6.,  7.,  9., 10., 11., 13., 14., 15.]]])

In [51]:
conv= torch.nn.Conv2d(in_channels=1,out_channels=1,kernel_size=(2,2),bias=False)
print(f'weight: {conv.weight.data}')
print(f'unfold(weight): {unfold(conv.weight.data)}, {unfold(conv.weight.data).shape}')

weight: tensor([[[[-0.4410, -0.1046],
          [ 0.4316,  0.4210]]]])
unfold(weight): tensor([[[-0.4410],
         [-0.1046],
         [ 0.4316],
         [ 0.4210]]]), torch.Size([1, 4, 1])


In [59]:
print(f'conv(img): {conv(img)}')
torch.bmm(img_u.transpose(2,1), unfold(conv.weight.data)).view(-1,1,3,3)

conv(img): tensor([[[[3.7269, 4.0339, 4.3409],
          [4.9549, 5.2619, 5.5690],
          [6.1830, 6.4900, 6.7970]]]], grad_fn=<ThnnConv2DBackward>)


tensor([[[[3.7269, 4.0339, 4.3409],
          [4.9549, 5.2619, 5.5690],
          [6.1830, 6.4900, 6.7970]]]])

## Convolution

In [112]:
from ay_torch import ConvLayer

In [32]:
img= torch.from_numpy(np.random.uniform(0,1,size=(1,1,8,8))).float()
img

tensor([[[[0.2839, 0.8808, 0.0139, 0.8287, 0.5301, 0.6634, 0.6912, 0.9814],
          [0.1733, 0.3598, 0.4103, 0.4994, 0.0770, 0.8473, 0.9285, 0.0275],
          [0.3501, 0.0781, 0.8500, 0.1458, 0.3770, 0.5725, 0.0959, 0.7596],
          [0.3627, 0.7821, 0.7763, 0.7251, 0.4945, 0.2048, 0.5201, 0.4572],
          [0.2111, 0.1763, 0.3066, 0.7494, 0.9515, 0.7900, 0.2000, 0.5780],
          [0.3230, 0.6924, 0.6222, 0.3055, 0.3279, 0.3511, 0.2773, 0.9002],
          [0.1326, 0.8908, 0.0514, 0.3623, 0.6731, 0.9318, 0.0854, 0.5532],
          [0.4673, 0.3573, 0.9547, 0.6804, 0.6719, 0.8438, 0.7647, 0.1715]]]])

In [33]:
# ConvLayer(1,3,3)(img)
# ConvLayer(1,1,3,transpose=True)(img)
print(ConvLayer(3,1,3,transpose=True)(ConvLayer(1,3,3)(img)))
print(img.shape, ConvLayer(1,3,3)(img).shape, ConvLayer(3,1,3,transpose=True)(ConvLayer(1,3,3)(img)).shape)

tensor([[[[1.0390, 0.8798, 0.0000, 0.1952, 1.0140, 0.0000, 0.0000, 1.3001],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.1778, 1.4182],
          [0.4135, 0.0000, 0.0000, 0.1114, 0.0000, 0.0000, 1.1794, 0.3791],
          [0.0000, 0.5827, 1.8779, 0.4145, 0.0000, 0.0203, 0.0000, 1.8402],
          [0.0000, 0.0000, 0.0000, 0.0000, 1.6081, 1.3924, 0.0000, 0.5100],
          [0.0000, 0.0000, 0.7051, 0.4953, 0.0000, 0.1407, 0.0000, 1.0345],
          [0.0000, 0.0000, 0.0998, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.3873, 1.3679, 0.8526, 0.6056, 0.1805, 1.2865, 1.7175, 0.3159]]]],
       grad_fn=<ReluBackward1>)
torch.Size([1, 1, 8, 8]) torch.Size([1, 3, 8, 8]) torch.Size([1, 1, 8, 8])


In [34]:
print(ConvLayer(1,3,3,stride=2)(img))
print(ConvLayer(3,1,3,stride=2,transpose=True)(ConvLayer(1,3,3,stride=2)(img)))
print(img.shape, ConvLayer(1,3,3,stride=2)(img).shape, ConvLayer(3,1,3,stride=2,transpose=True)(ConvLayer(1,3,3,stride=2)(img)).shape)

tensor([[[[1.2139, 0.0000, 0.1058, 0.0000],
          [0.3911, 0.0000, 1.5457, 0.0000],
          [0.8706, 0.1721, 0.0000, 0.0000],
          [1.2724, 0.0000, 0.0813, 0.0000]],

         [[0.9741, 0.0000, 1.2964, 0.0000],
          [0.6860, 0.2006, 0.0000, 0.0000],
          [0.0000, 0.0000, 1.2277, 0.2832],
          [0.0000, 0.0000, 1.4465, 0.0000]],

         [[0.0000, 0.9920, 0.0000, 1.9861],
          [0.0000, 0.0000, 0.7416, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.9116, 0.0000, 2.2223]]]], grad_fn=<ReluBackward1>)
tensor([[[[0.0000, 0.0000, 0.0000, 0.4904, 0.0000, 0.0000, 0.0000, 2.1680],
          [0.2877, 0.6108, 0.1776, 0.6291, 0.0000, 4.0315, 0.9366, 3.0488],
          [0.2264, 1.2161, 0.3334, 0.0000, 0.0000, 0.0000, 0.0000, 0.0207],
          [0.0000, 1.0006, 0.0000, 0.6195, 0.0000, 0.5516, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5167, 0.0000, 0.0000],
          [0.0000, 0.3972, 0.0000, 0.0111, 0.0000, 0.0000, 0

## ResBlock

In [97]:
from ay_torch import TResBlock

In [98]:
print(img.shape)
torchinfo.summary(TResBlock(1,1,3),img.shape)
# TResBlock(1,1,3)(img)

torch.Size([1, 1, 8, 8])


Layer (type:depth-idx)                   Output Shape              Param #
TResBlock                                --                        --
├─Sequential: 1-1                        [1, 3, 8, 8]              --
│    └─Sequential: 2-1                   [1, 3, 8, 8]              --
│    │    └─Conv2d: 3-1                  [1, 3, 8, 8]              27
│    │    └─BatchNorm2d: 3-2             [1, 3, 8, 8]              6
│    │    └─ReLU: 3-3                    [1, 3, 8, 8]              --
│    └─Sequential: 2-2                   [1, 3, 8, 8]              --
│    │    └─Conv2d: 3-4                  [1, 3, 8, 8]              81
│    │    └─BatchNorm2d: 3-5             [1, 3, 8, 8]              6
├─Sequential: 1-2                        [1, 3, 8, 8]              --
│    └─Sequential: 2-3                   [1, 3, 8, 8]              --
│    │    └─Conv2d: 3-6                  [1, 3, 8, 8]              3
│    │    └─BatchNorm2d: 3-7             [1, 3, 8, 8]              6
├─ReLU: 1-3        

In [99]:
img2= TResBlock(1,1,3)(img)
print(img2.shape)
torchinfo.summary(TResBlock(1,3,1,transpose=True),img2.shape)
# TResBlock(1,3,1)(img2)

torch.Size([1, 3, 8, 8])


Layer (type:depth-idx)                   Output Shape              Param #
TResBlock                                --                        --
├─Sequential: 1-1                        [1, 1, 8, 8]              --
│    └─Sequential: 2-1                   [1, 1, 8, 8]              --
│    │    └─ConvTranspose2d: 3-1         [1, 1, 8, 8]              27
│    │    └─BatchNorm2d: 3-2             [1, 1, 8, 8]              2
│    │    └─ReLU: 3-3                    [1, 1, 8, 8]              --
│    └─Sequential: 2-2                   [1, 1, 8, 8]              --
│    │    └─ConvTranspose2d: 3-4         [1, 1, 8, 8]              9
│    │    └─BatchNorm2d: 3-5             [1, 1, 8, 8]              2
├─Sequential: 1-2                        [1, 1, 8, 8]              --
│    └─Sequential: 2-3                   [1, 1, 8, 8]              --
│    │    └─ConvTranspose2d: 3-6         [1, 1, 8, 8]              3
│    │    └─BatchNorm2d: 3-7             [1, 1, 8, 8]              2
├─ReLU: 1-3         

In [102]:
print(img.shape)
torchinfo.summary(TResBlock(1,1,3,stride=2),img.shape)
# TResBlock(1,1,3,stride=2)(img)
# rb= TResBlock(1,1,3,stride=2)
# rb.convpath(img).shape
# rb.idpath(img).shape
# rb

torch.Size([1, 1, 8, 8])


Layer (type:depth-idx)                   Output Shape              Param #
TResBlock                                --                        --
├─Sequential: 1-1                        [1, 3, 4, 4]              --
│    └─Sequential: 2-1                   [1, 3, 4, 4]              --
│    │    └─Conv2d: 3-1                  [1, 3, 4, 4]              27
│    │    └─BatchNorm2d: 3-2             [1, 3, 4, 4]              6
│    │    └─ReLU: 3-3                    [1, 3, 4, 4]              --
│    └─Sequential: 2-2                   [1, 3, 4, 4]              --
│    │    └─Conv2d: 3-4                  [1, 3, 4, 4]              81
│    │    └─BatchNorm2d: 3-5             [1, 3, 4, 4]              6
├─Sequential: 1-2                        [1, 3, 4, 4]              --
│    └─AvgPool2d: 2-3                    [1, 1, 4, 4]              --
│    └─Sequential: 2-4                   [1, 3, 4, 4]              --
│    │    └─Conv2d: 3-6                  [1, 3, 4, 4]              3
│    │    └─BatchN

In [111]:
img2= TResBlock(1,1,3,stride=2)(img)
print(img2.shape)
torchinfo.summary(TResBlock(1,3,1,stride=2,transpose=True),img2.shape)
# TResBlock(1,3,1,stride=2,transpose=True)(img2)
# rb= TResBlock(1,3,1,stride=2,transpose=True,upsample_first=True)
# print(rb.convpath(img2).shape)
# print(rb.idpath(img2).shape)
# rb

torch.Size([1, 3, 4, 4])


Layer (type:depth-idx)                   Output Shape              Param #
TResBlock                                --                        --
├─Sequential: 1-1                        [1, 1, 8, 8]              --
│    └─Sequential: 2-1                   [1, 1, 8, 8]              --
│    │    └─ConvTranspose2d: 3-1         [1, 1, 8, 8]              27
│    │    └─BatchNorm2d: 3-2             [1, 1, 8, 8]              2
│    │    └─ReLU: 3-3                    [1, 1, 8, 8]              --
│    └─Sequential: 2-2                   [1, 1, 8, 8]              --
│    │    └─ConvTranspose2d: 3-4         [1, 1, 8, 8]              9
│    │    └─BatchNorm2d: 3-5             [1, 1, 8, 8]              2
├─Sequential: 1-2                        [1, 1, 8, 8]              --
│    └─Sequential: 2-3                   [1, 1, 4, 4]              --
│    │    └─ConvTranspose2d: 3-6         [1, 1, 4, 4]              3
│    │    └─BatchNorm2d: 3-7             [1, 1, 4, 4]              2
│    └─Upsample: 2-4

## ResNet without pooling

In [19]:
from ay_torch import TResBlock, TResNet, TNoop

In [35]:
img= torch.from_numpy(np.random.uniform(0,1,size=(2,1,8,8))).float()
img

tensor([[[[0.6807, 0.6319, 0.1488, 0.2314, 0.1645, 0.2275, 0.8301, 0.4215],
          [0.8867, 0.2966, 0.9344, 0.6837, 0.9192, 0.1543, 0.9079, 0.7589],
          [0.8372, 0.8294, 0.2528, 0.7335, 0.8887, 0.3479, 0.5266, 0.9375],
          [0.7423, 0.2302, 0.8697, 0.0619, 0.1730, 0.6311, 0.1207, 0.5635],
          [0.4931, 0.7887, 0.2703, 0.6939, 0.4062, 0.9014, 0.7710, 0.3900],
          [0.5141, 0.9686, 0.1900, 0.2002, 0.1720, 0.5338, 0.1929, 0.7724],
          [0.9152, 0.2015, 0.0836, 0.3036, 0.2042, 0.3879, 0.0426, 0.1872],
          [0.9524, 0.5482, 0.6289, 0.8618, 0.2832, 0.5301, 0.9718, 0.6986]]],


        [[[0.4640, 0.2365, 0.9946, 0.7539, 0.0234, 0.3142, 0.6843, 0.0659],
          [0.1769, 0.6768, 0.4790, 0.5934, 0.4234, 0.1729, 0.3388, 0.0212],
          [0.4978, 0.1483, 0.8279, 0.4131, 0.3294, 0.4945, 0.3556, 0.1566],
          [0.6589, 0.4510, 0.2039, 0.9783, 0.6589, 0.1906, 0.8111, 0.5160],
          [0.6891, 0.9937, 0.1032, 0.4532, 0.0265, 0.9145, 0.3716, 0.3260],
        

In [42]:
print(img.shape)
# TResNet(TResBlock,1,[2,2,2,2],in_channels=1,out_channels=1)(img)
print(torchinfo.summary(TResNet(TResBlock,1,[2,2,2,2],in_channels=1,out_channels=1,rnpool=TNoop),img.shape))
# print(torchinfo.summary(TResBlock(1,1,3,stride=2,pool=TNoop),img.shape))
# TResBlock(1,1,3)(img)

torch.Size([2, 1, 8, 8])
Layer (type:depth-idx)                        Output Shape              Param #
TResNet                                       --                        --
├─Sequential: 1-1                             [2, 32, 8, 8]             --
│    └─Conv2d: 2-1                            [2, 32, 8, 8]             288
│    └─BatchNorm2d: 2-2                       [2, 32, 8, 8]             64
│    └─ReLU: 2-3                              [2, 32, 8, 8]             --
├─Sequential: 1-2                             [2, 32, 4, 4]             --
│    └─Conv2d: 2-4                            [2, 32, 4, 4]             9,216
│    └─BatchNorm2d: 2-5                       [2, 32, 4, 4]             64
│    └─ReLU: 2-6                              [2, 32, 4, 4]             --
├─Sequential: 1-3                             [2, 64, 2, 2]             --
│    └─Conv2d: 2-7                            [2, 64, 2, 2]             18,432
│    └─BatchNorm2d: 2-8                       [2, 64, 2, 2]   

# Torch functions

## Softmax and clamp

In [143]:
img= torch.from_numpy(np.random.uniform(-1,1,size=(1,1,5,5)))
img

tensor([[[[ 0.1590, -0.4883,  0.0481, -0.2388, -0.4310],
          [-0.2900,  0.2253, -0.2798,  0.2368,  0.6311],
          [-0.4764,  0.6701,  0.8507,  0.3041,  0.8848],
          [ 0.0155, -0.5271, -0.7959,  0.2696,  0.9397],
          [ 0.2211,  0.3824, -0.3784,  0.1784,  0.5068]]]],
       dtype=torch.float64)

In [144]:
torch.sigmoid(img)

tensor([[[[0.5397, 0.3803, 0.5120, 0.4406, 0.3939],
          [0.4280, 0.5561, 0.4305, 0.5589, 0.6527],
          [0.3831, 0.6615, 0.7007, 0.5754, 0.7078],
          [0.5039, 0.3712, 0.3109, 0.5670, 0.7190],
          [0.5551, 0.5945, 0.4065, 0.5445, 0.6240]]]], dtype=torch.float64)

In [145]:
torch.clamp(img,0,1)

tensor([[[[0.1590, 0.0000, 0.0481, 0.0000, 0.0000],
          [0.0000, 0.2253, 0.0000, 0.2368, 0.6311],
          [0.0000, 0.6701, 0.8507, 0.3041, 0.8848],
          [0.0155, 0.0000, 0.0000, 0.2696, 0.9397],
          [0.2211, 0.3824, 0.0000, 0.1784, 0.5068]]]], dtype=torch.float64)

# Implementing convolution from scratch
Ref. https://stackoverflow.com/questions/43086557/convolve2d-just-by-using-numpy

## Convolution (N-batch=1, N-inchannels=1, N-outchannels=1)

In [108]:
img= np.round(np.random.uniform(0,1,size=(6,9)),2)
w= np.round(np.random.uniform(-1,1,size=(3,3)),2)
print(f'img= {img}')
print(f'w= {w}')

img= [[0.08 0.93 0.67 0.68 0.33 0.15 0.16 0.76 0.28]
 [0.48 0.57 0.15 0.68 0.49 0.69 0.39 0.99 0.54]
 [0.8  0.42 0.61 0.07 0.7  0.22 0.45 0.26 0.88]
 [0.11 0.35 0.65 0.08 0.65 0.52 0.23 0.86 0.9 ]
 [0.06 0.68 0.63 0.34 0.46 0.96 0.3  0.82 0.78]
 [0.22 0.88 0.9  0.02 0.26 0.14 0.57 0.45 0.23]]
w= [[-0.92  0.33 -0.08]
 [-0.4   0.85 -0.37]
 [-0.91  0.31  0.42]]


### Ground truth: torch.conv2d

In [109]:
img_torch= torch.from_numpy(img).reshape((1,1,)+img.shape).float()
w_torch= torch.from_numpy(w).reshape((1,1,)+w.shape).float()
torch.nn.functional.conv2d(img_torch, w_torch)

tensor([[[[ 0.0751, -1.2047, -0.3211, -0.3938, -0.4005, -0.4083,  0.6076],
          [-0.1728, -0.2881, -0.6900,  0.3136, -0.8475, -0.4277,  0.0754],
          [-0.2124, -0.0885, -1.3013,  0.7135, -0.5055, -0.8416,  0.2162],
          [ 0.7338, -0.4896, -1.4604,  0.1203,  0.1224, -0.6654,  0.0060]]]])

### With numpy stride_tricks.as_strided and einsum

In [110]:
view_shape= tuple(np.subtract(img.shape, w.shape)+1)+w.shape
print(f'view_shape= {view_shape}')
strides= img.strides+img.strides
print(f'strides= {strides}')
sub_matrices= np.lib.stride_tricks.as_strided(img, view_shape, strides)
# print(f'sub_matrices= {sub_matrices}')
conv1= (sub_matrices*w).sum(axis=(2,3))
print(f'conv1= {conv1}')
conv2= np.einsum('ij,klij->kl', w, sub_matrices)
print(f'conv2= {conv2}')

view_shape= (4, 7, 3, 3)
strides= (72, 8, 72, 8)
conv1= [[ 0.0751 -1.2047 -0.3211 -0.3938 -0.4005 -0.4083  0.6076]
 [-0.1728 -0.2881 -0.69    0.3136 -0.8475 -0.4277  0.0754]
 [-0.2124 -0.0885 -1.3013  0.7135 -0.5055 -0.8416  0.2162]
 [ 0.7338 -0.4896 -1.4604  0.1203  0.1224 -0.6654  0.006 ]]
conv2= [[ 0.0751 -1.2047 -0.3211 -0.3938 -0.4005 -0.4083  0.6076]
 [-0.1728 -0.2881 -0.69    0.3136 -0.8475 -0.4277  0.0754]
 [-0.2124 -0.0885 -1.3013  0.7135 -0.5055 -0.8416  0.2162]
 [ 0.7338 -0.4896 -1.4604  0.1203  0.1224 -0.6654  0.006 ]]


### With Torch

In [111]:
img_torch= torch.from_numpy(img).reshape((1,1,)+img.shape).float()
w_torch= torch.from_numpy(w).reshape((1,1,)+w.shape).float()
view_shape= img_torch.shape[:2]+tuple(np.subtract(img_torch.shape[2:],w_torch.shape[2:])+1)+w_torch.shape[2:]
print(f'view_shape= {view_shape}')
strides= img_torch.stride()+img_torch.stride()[2:]
print(f'strides= {strides}')
sub_matrices= torch.as_strided(img_torch, view_shape, strides)
# print(f'sub_matrices= {sub_matrices}')
conv1= (sub_matrices*w_torch).sum(axis=(4,5))
print(f'conv1= {conv1}')
conv2= torch.einsum('klij,klmnij->klmn', w_torch, sub_matrices)
print(f'conv2= {conv2}')

view_shape= torch.Size([1, 1, 4, 7, 3, 3])
strides= (54, 54, 9, 1, 9, 1)
conv1= tensor([[[[ 0.0751, -1.2047, -0.3211, -0.3938, -0.4005, -0.4083,  0.6076],
          [-0.1728, -0.2881, -0.6900,  0.3136, -0.8475, -0.4277,  0.0754],
          [-0.2124, -0.0885, -1.3013,  0.7135, -0.5055, -0.8416,  0.2162],
          [ 0.7338, -0.4896, -1.4604,  0.1203,  0.1224, -0.6654,  0.0060]]]])
conv2= tensor([[[[ 0.0751, -1.2047, -0.3211, -0.3938, -0.4005, -0.4083,  0.6076],
          [-0.1728, -0.2881, -0.6900,  0.3136, -0.8475, -0.4277,  0.0754],
          [-0.2124, -0.0885, -1.3013,  0.7135, -0.5055, -0.8416,  0.2162],
          [ 0.7338, -0.4896, -1.4604,  0.1203,  0.1224, -0.6654,  0.0060]]]])


## Convolution (N-batch=1, N-inchannels=2, N-outchannels=3)
Note: The following code works with n_bch>=2.

In [153]:
n_bch,in_ch,out_ch,ks= 1,2,3,3
img= np.round(np.random.uniform(0,1,size=(n_bch,in_ch,6,9)),2)
w= np.round(np.random.uniform(-1,1,size=(out_ch,in_ch,ks,ks)),2)
print(f'img= {img}')
print(f'w= {w}')

img= [[[[0.58 0.38 0.12 0.17 0.97 0.18 0.95 0.95 0.27]
   [0.09 0.86 0.63 0.99 0.84 0.24 0.21 0.26 0.21]
   [0.89 0.43 0.9  0.75 0.87 0.16 0.41 0.08 0.11]
   [0.16 0.81 0.15 0.56 0.35 0.11 0.41 0.9  0.09]
   [0.73 0.43 0.98 0.79 0.52 0.54 0.15 0.25 0.19]
   [0.99 0.19 0.04 0.77 0.36 0.47 0.14 0.52 0.15]]

  [[0.89 0.98 0.89 0.14 0.78 0.14 0.95 0.95 0.73]
   [0.12 0.13 0.87 0.72 0.27 0.96 0.91 0.84 0.14]
   [0.37 0.56 0.06 0.5  0.27 0.07 0.57 0.86 0.44]
   [0.46 0.13 0.26 0.31 0.9  0.86 0.93 0.24 0.58]
   [0.84 0.62 0.58 0.91 0.86 0.59 0.16 0.31 0.32]
   [0.39 0.44 0.67 0.37 0.96 0.83 0.26 0.91 0.58]]]]
w= [[[[-1.   -0.12 -0.24]
   [ 0.5   0.92  0.81]
   [ 0.47  0.18 -0.04]]

  [[-0.11  0.6   0.16]
   [-0.19  0.49 -0.37]
   [-0.09  0.44  0.28]]]


 [[[ 0.5  -0.2   0.45]
   [-0.49 -0.34  0.54]
   [-0.6   0.16 -0.82]]

  [[ 0.42 -0.53 -0.91]
   [-0.79  0.3   0.42]
   [-0.93 -0.9  -0.54]]]


 [[[-0.69 -0.89  0.13]
   [-0.94 -0.65  0.37]
   [-0.46  0.03 -0.45]]

  [[-0.3  -0.93  0.54]
   [ 

### Ground truth: torch.conv2d

In [154]:
# print(torch.nn.Conv2d(2,3,bias=False,kernel_size=(3,3)).weight.shape)
img_torch= torch.from_numpy(img).float()
w_torch= torch.from_numpy(w).float()
conv_gt= torch.nn.functional.conv2d(img_torch, w_torch)
print(f'conv_gt= {conv_gt}')

conv_gt= tensor([[[[ 1.7332,  2.4102,  2.5449,  1.8440,  0.4113,  1.2922,  0.8077],
          [ 1.9147,  1.4130,  1.9298,  1.3351,  1.1848,  1.2055,  1.1881],
          [ 0.7150,  1.1221,  0.9520,  0.8854,  0.1931,  1.9831,  1.2294],
          [ 2.1038,  1.2396,  2.1700,  2.3855,  1.6488,  1.1085,  0.7497]],

         [[-2.4579, -1.1591, -2.2823, -2.4235, -1.1120, -2.3014, -2.7134],
          [-1.6237, -2.0818, -1.0713, -3.0737, -3.6110, -2.8510, -1.7890],
          [-2.7452, -2.2581, -2.3141, -2.4440, -1.6706, -2.3933, -2.0621],
          [-2.0524, -1.7402, -2.7161, -3.8014, -3.4150, -2.6239, -1.6516]],

         [[-2.0643, -2.1252, -0.7881, -2.5149, -1.4129, -0.2671, -1.1814],
          [-0.6381, -2.0971, -2.4688, -1.4834, -0.8247, -0.7548, -0.6436],
          [-1.5008, -0.9920, -1.2218, -1.5451,  0.3733,  0.1928, -1.1368],
          [-0.6915, -0.7645, -0.4683, -0.9015, -0.1823, -0.8607, -0.6381]]]])


### With numpy stride_tricks.as_strided and einsum

In [155]:
view_shape= img.shape[:2]+tuple(np.subtract(img.shape[2:], w.shape[-2:])+1)+w.shape[-2:]
print(f'view_shape= {view_shape}')
strides= img.strides+img.strides[2:]
print(f'strides= {strides}')
sub_matrices= np.lib.stride_tricks.as_strided(img, view_shape, strides)
print(f'sub_matrices.shape= {sub_matrices.shape}')
print(f'w.shape= {w.shape}')
# print(f'sub_matrices= {sub_matrices}')
conv_np= np.einsum('ijkl,bjmnkl->bimn', w, sub_matrices)
print(f'conv_np= {conv_np}, {torch.all(torch.isclose(conv_gt,torch.from_numpy(conv_np).float()))}')

view_shape= (1, 2, 4, 7, 3, 3)
strides= (864, 432, 72, 8, 72, 8)
sub_matrices.shape= (1, 2, 4, 7, 3, 3)
w.shape= (3, 2, 3, 3)
conv_np= [[[[ 1.7332  2.4102  2.5449  1.844   0.4113  1.2922  0.8077]
   [ 1.9147  1.413   1.9298  1.3351  1.1848  1.2055  1.1881]
   [ 0.715   1.1221  0.952   0.8854  0.1931  1.9831  1.2294]
   [ 2.1038  1.2396  2.17    2.3855  1.6488  1.1085  0.7497]]

  [[-2.4579 -1.1591 -2.2823 -2.4235 -1.112  -2.3014 -2.7134]
   [-1.6237 -2.0818 -1.0713 -3.0737 -3.611  -2.851  -1.789 ]
   [-2.7452 -2.2581 -2.3141 -2.444  -1.6706 -2.3933 -2.0621]
   [-2.0524 -1.7402 -2.7161 -3.8014 -3.415  -2.6239 -1.6516]]

  [[-2.0643 -2.1252 -0.7881 -2.5149 -1.4129 -0.2671 -1.1814]
   [-0.6381 -2.0971 -2.4688 -1.4834 -0.8247 -0.7548 -0.6436]
   [-1.5008 -0.992  -1.2218 -1.5451  0.3733  0.1928 -1.1368]
   [-0.6915 -0.7645 -0.4683 -0.9015 -0.1823 -0.8607 -0.6381]]]], True


### With Torch

In [156]:
img_torch= torch.from_numpy(img).float()
w_torch= torch.from_numpy(w).float()
view_shape= img_torch.shape[:2]+tuple(np.subtract(img_torch.shape[2:],w_torch.shape[-2:])+1)+w_torch.shape[-2:]
print(f'view_shape= {view_shape}')
strides= img_torch.stride()+img_torch.stride()[2:]
print(f'strides= {strides}')
sub_matrices= torch.as_strided(img_torch, view_shape, strides)
print(f'sub_matrices.shape= {sub_matrices.shape}')
# print(f'sub_matrices= {sub_matrices}')
conv_tch= torch.einsum('ijkl,bjmnkl->bimn', w_torch, sub_matrices)
print(f'conv_tch= {conv_tch}, {torch.all(torch.isclose(conv_gt,conv_tch))}')

view_shape= torch.Size([1, 2, 4, 7, 3, 3])
strides= (108, 54, 9, 1, 9, 1)
sub_matrices.shape= torch.Size([1, 2, 4, 7, 3, 3])
conv_tch= tensor([[[[ 1.7332,  2.4102,  2.5449,  1.8440,  0.4113,  1.2922,  0.8077],
          [ 1.9147,  1.4130,  1.9298,  1.3351,  1.1848,  1.2055,  1.1881],
          [ 0.7150,  1.1221,  0.9520,  0.8854,  0.1931,  1.9831,  1.2294],
          [ 2.1038,  1.2396,  2.1700,  2.3855,  1.6488,  1.1085,  0.7497]],

         [[-2.4579, -1.1591, -2.2823, -2.4235, -1.1120, -2.3014, -2.7134],
          [-1.6237, -2.0818, -1.0713, -3.0737, -3.6110, -2.8510, -1.7890],
          [-2.7452, -2.2581, -2.3141, -2.4440, -1.6706, -2.3933, -2.0621],
          [-2.0524, -1.7402, -2.7161, -3.8014, -3.4150, -2.6239, -1.6516]],

         [[-2.0643, -2.1252, -0.7881, -2.5149, -1.4129, -0.2671, -1.1814],
          [-0.6381, -2.0971, -2.4688, -1.4834, -0.8247, -0.7548, -0.6436],
          [-1.5008, -0.9920, -1.2218, -1.5451,  0.3733,  0.1928, -1.1368],
          [-0.6915, -0.7645, -0.4683

## Pixel-variant kernel (N-batch=1, N-inchannels=1, N-outchannels=1)

In [112]:
kernel_shape= (3,3)
img= np.round(np.random.uniform(0,1,size=(6,9)),2)
out_shape= tuple(np.subtract(img.shape, kernel_shape)+1)
w= np.round(np.random.uniform(-1,1,size=out_shape+kernel_shape),2)
print(f'img= {img}')
print(f'w= {w[0:1,0:1,:,:]}..., {w.shape}')

img= [[0.78 0.79 0.32 0.44 0.79 0.26 0.59 0.34 0.38]
 [0.94 0.62 0.43 0.03 0.82 0.45 0.63 0.64 0.43]
 [0.67 0.69 0.81 0.63 0.37 0.91 0.35 0.36 0.56]
 [0.02 0.74 0.88 0.68 0.51 0.15 0.41 0.68 0.29]
 [0.58 0.9  0.23 0.1  0.26 0.06 0.35 0.13 0.84]
 [0.57 0.67 0.11 0.6  0.3  0.58 0.75 0.15 0.06]]
w= [[[[ 0.12  0.07  0.34]
   [ 0.07  0.27  0.67]
   [ 0.55 -0.11  0.77]]]]..., (4, 7, 3, 3)


### With numpy stride_tricks.as_strided and einsum

In [113]:
view_shape= out_shape+kernel_shape
print(f'view_shape= {view_shape}')
strides= img.strides+img.strides
print(f'strides= {strides}')
sub_matrices= np.lib.stride_tricks.as_strided(img, view_shape, strides)
print(f'sub_matrices.shape= {sub_matrices.shape}')
# print(f'sub_matrices= {sub_matrices}')
# print(f'sub_matrices*w= {sub_matrices*w}')
res1= (sub_matrices*w).sum(axis=(2,3))
print(f'res1= {res1}')
res2= np.einsum('ijkl,ijkl->ij', w, sub_matrices)
print(f'res2= {res2}')

view_shape= (4, 7, 3, 3)
strides= (72, 8, 72, 8)
sub_matrices.shape= (4, 7, 3, 3)
res1= [[ 1.6953 -2.4214  0.5561  0.3529 -0.6678  0.8172  0.7055]
 [-1.5766 -1.3174 -1.0877  1.7223  1.1279 -1.3055  1.6701]
 [ 0.2433 -1.561   0.8805 -0.0295  0.9864  0.448   0.4326]
 [-1.2691 -0.5238 -0.5407 -1.1373 -0.2999  1.304  -1.7477]]
res2= [[ 1.6953 -2.4214  0.5561  0.3529 -0.6678  0.8172  0.7055]
 [-1.5766 -1.3174 -1.0877  1.7223  1.1279 -1.3055  1.6701]
 [ 0.2433 -1.561   0.8805 -0.0295  0.9864  0.448   0.4326]
 [-1.2691 -0.5238 -0.5407 -1.1373 -0.2999  1.304  -1.7477]]


### With Torch

In [114]:
img_torch= torch.from_numpy(img).reshape((1,1,)+img.shape).float()
w_torch= torch.from_numpy(w).reshape((1,1,)+w.shape).float()
view_shape= (1,1,)+out_shape+kernel_shape
print(f'view_shape= {view_shape}')
strides= img_torch.stride()+img_torch.stride()[2:]
print(f'strides= {strides}')
sub_matrices= torch.as_strided(img_torch, view_shape, strides)
print(f'sub_matrices.shape= {sub_matrices.shape}')
# print(f'sub_matrices= {sub_matrices}')
res1= (sub_matrices*w_torch).sum(axis=(4,5))
print(f'res1= {res1}')
res2= torch.einsum('ijklmn,ijklmn->ijkl', w_torch, sub_matrices)
print(f'res2= {res2}')

view_shape= (1, 1, 4, 7, 3, 3)
strides= (54, 54, 9, 1, 9, 1)
sub_matrices.shape= torch.Size([1, 1, 4, 7, 3, 3])
res1= tensor([[[[ 1.6953, -2.4214,  0.5561,  0.3529, -0.6678,  0.8172,  0.7055],
          [-1.5766, -1.3174, -1.0877,  1.7223,  1.1279, -1.3055,  1.6701],
          [ 0.2433, -1.5610,  0.8805, -0.0295,  0.9864,  0.4480,  0.4326],
          [-1.2691, -0.5238, -0.5407, -1.1373, -0.2999,  1.3040, -1.7477]]]])
res2= tensor([[[[ 1.6953, -2.4214,  0.5561,  0.3529, -0.6678,  0.8172,  0.7055],
          [-1.5766, -1.3174, -1.0877,  1.7223,  1.1279, -1.3055,  1.6701],
          [ 0.2433, -1.5610,  0.8805, -0.0295,  0.9864,  0.4480,  0.4326],
          [-1.2691, -0.5238, -0.5407, -1.1373, -0.2999,  1.3040, -1.7477]]]])


## Pixel-variant kernel (N-batch=1, N-inchannels=2, N-outchannels=3)
Note: The following code works with N-batch>=2.0.

In [192]:
n_bch,in_ch,out_ch,ks= 1,2,3,3
img= np.round(np.random.uniform(0,1,size=(n_bch,in_ch,6,9)),2)
kernel_shape= (ks,ks)
out_shape= tuple(np.subtract(img.shape[2:], kernel_shape)+1)
w= np.round(np.random.uniform(-1,1,size=(out_ch,in_ch)+out_shape+kernel_shape),2)
print(f'img= {img}, {img.shape}')
print(f'w= {w[0:1,0:1,0:1,0:1,:,:]}..., {w.shape}')

img= [[[[0.98 0.13 0.47 0.79 0.96 0.87 0.65 0.45 0.27]
   [0.22 0.94 1.   0.39 0.41 0.7  0.98 0.95 0.64]
   [0.82 0.94 0.75 0.48 0.7  0.42 0.91 0.3  0.82]
   [0.94 0.42 0.02 0.8  0.63 0.36 0.68 0.71 0.93]
   [0.44 0.55 0.94 0.22 0.06 0.9  0.69 0.86 0.24]
   [0.86 0.46 0.65 0.13 0.25 0.69 0.26 0.54 0.14]]

  [[0.11 0.43 0.89 0.6  0.   0.24 0.92 0.36 0.05]
   [0.78 0.69 0.43 0.87 0.3  0.19 0.06 0.84 0.6 ]
   [0.09 0.88 0.89 0.15 0.66 0.97 0.47 0.97 0.4 ]
   [0.52 0.31 0.89 0.1  0.22 0.37 0.22 0.92 0.85]
   [0.65 0.21 0.31 0.53 0.29 0.66 0.13 0.67 0.48]
   [0.33 0.31 0.56 0.84 0.41 0.45 0.82 0.6  0.61]]]], (1, 2, 6, 9)
w= [[[[[[ 0.36  0.26  0.67]
     [-0.35  0.67  0.49]
     [-0.13  0.26  0.62]]]]]]..., (3, 2, 4, 7, 3, 3)


### Test: comparison with torch.conv2d

In [189]:
w_base= np.round(np.random.uniform(-1,1,size=(out_ch,in_ch)+(1,1)+kernel_shape),2)
w= np.repeat(np.repeat(w_base,out_shape[1],axis=3),out_shape[0],axis=2)
print(f'img= {img}, {img.shape}')
print(f'w= {w[0:1,0:1,0:1,0:1,:,:]}..., {w.shape}, {w_base.shape}')
img_torch= torch.from_numpy(img).float()
w_base_torch= torch.from_numpy(w_base).reshape(w_base.shape[:2]+w_base.shape[4:]).float()
# print(f'w_base_torch= {w_base_torch}..., {w_base_torch.shape}')
res_gt= torch.nn.functional.conv2d(img_torch, w_base_torch)
print(f'res_gt= {res_gt}')

img= [[[[0.72 0.12 0.27 0.56 0.25 0.86 0.9  0.18 0.46]
   [0.56 0.12 0.72 0.28 0.79 0.21 0.61 0.75 0.96]
   [0.3  0.67 0.49 0.28 0.84 0.06 0.42 0.7  0.43]
   [0.13 0.62 0.24 0.1  0.05 0.59 0.43 0.41 0.62]
   [0.69 0.75 0.12 0.09 0.62 0.95 0.78 0.6  0.88]
   [0.15 0.86 0.43 0.78 0.6  0.18 0.25 0.34 0.63]]

  [[0.34 0.19 0.52 0.19 0.25 0.97 0.71 0.98 0.97]
   [0.73 0.56 0.74 0.81 0.37 0.48 0.4  0.39 0.32]
   [0.78 0.37 0.78 0.4  0.39 0.67 0.27 0.3  0.17]
   [0.15 0.81 0.4  0.11 0.38 0.61 0.78 0.37 0.58]
   [0.56 0.47 0.54 0.11 0.39 0.94 0.16 0.75 0.12]
   [0.66 0.43 0.54 0.02 0.41 0.49 0.41 0.23 0.15]]]], (1, 2, 6, 9)
w= [[[[[[ 0.9   0.67  0.26]
     [ 0.59  0.43 -0.64]
     [-0.07  0.92 -0.77]]]]]]..., (3, 2, 4, 7, 3, 3), (3, 2, 1, 1, 3, 3)
res_gt= tensor([[[[-0.3094, -0.5318, -0.6258,  0.2221, -1.0388, -1.1220, -1.0099],
          [-1.3240, -0.4891, -0.8321, -1.0456,  0.1132, -0.8761,  0.5982],
          [-0.1455, -0.1262, -0.7849, -1.3490, -0.5358,  0.0394, -0.5605],
          [ 0.378

### With numpy stride_tricks.as_strided and einsum

In [193]:
view_shape= img.shape[:2]+out_shape+kernel_shape
print(f'view_shape= {view_shape}')
strides= img.strides+img.strides[2:]
print(f'strides= {strides}')
sub_matrices= np.lib.stride_tricks.as_strided(img, view_shape, strides)
print(f'sub_matrices.shape= {sub_matrices.shape}')
# print(f'sub_matrices= {sub_matrices}')
# print(f'sub_matrices*w= {sub_matrices*w}')
res_np= np.einsum('ijmnkl,bjmnkl->bimn', w, sub_matrices)
print(f'res_np= {res_np}')
# print(f'res_np= {res_np}, {torch.all(torch.isclose(res_gt,torch.from_numpy(res_np).float()))}')

view_shape= (1, 2, 4, 7, 3, 3)
strides= (864, 432, 72, 8, 72, 8)
sub_matrices.shape= (1, 2, 4, 7, 3, 3)
res_np= [[[[ 3.0778  1.7253 -0.5113 -0.0817 -0.581  -1.9058  0.7106]
   [-1.9801  1.1653  0.3585 -1.7316  1.3211  0.6694  0.5069]
   [ 0.6119  0.563  -0.7654  2.5973 -1.971   0.6882  0.8126]
   [ 0.7336  0.9963  1.6773 -1.0295  1.2635  2.6903 -0.332 ]]

  [[-1.364  -0.0372  1.4103 -0.1755  1.194  -0.2636  0.3186]
   [-1.9607  0.4     1.9175 -1.7362  0.4124  1.0521 -0.605 ]
   [-1.7676  2.4814  2.3001  0.1844 -0.6243  2.2404  0.7621]
   [-1.7687 -0.2826  0.0179 -0.3561  0.365   0.8226  0.6107]]

  [[-0.4571  1.1342  0.5213 -0.5604 -1.0119  0.4766  0.1784]
   [ 0.0692 -0.2373 -1.4442 -0.3252 -2.1491  2.8538  0.116 ]
   [-2.1918  0.3397  1.0173 -0.2993 -2.663   0.7206  0.1622]
   [-0.5263  1.0427  0.5387  0.6553 -1.3975  0.8994 -1.7948]]]]


### With Torch

In [194]:
img_torch= torch.from_numpy(img).float()
w_torch= torch.from_numpy(w).float()
view_shape= img_torch.shape[:2]+out_shape+kernel_shape
print(f'view_shape= {view_shape}')
strides= img_torch.stride()+img_torch.stride()[2:]
print(f'strides= {strides}')
sub_matrices= torch.as_strided(img_torch, view_shape, strides)
print(f'sub_matrices.shape= {sub_matrices.shape}')
# print(f'sub_matrices= {sub_matrices}')
res_tch= torch.einsum('ijmnkl,bjmnkl->bimn', w_torch, sub_matrices)
print(f'res_tch= {res_tch}, {torch.all(torch.isclose(res_tch,torch.from_numpy(res_np).float()))}')

view_shape= torch.Size([1, 2, 4, 7, 3, 3])
strides= (108, 54, 9, 1, 9, 1)
sub_matrices.shape= torch.Size([1, 2, 4, 7, 3, 3])
res_tch= tensor([[[[ 3.0778,  1.7253, -0.5113, -0.0817, -0.5810, -1.9058,  0.7106],
          [-1.9801,  1.1653,  0.3585, -1.7316,  1.3211,  0.6694,  0.5069],
          [ 0.6119,  0.5630, -0.7654,  2.5973, -1.9710,  0.6882,  0.8126],
          [ 0.7336,  0.9963,  1.6773, -1.0295,  1.2635,  2.6903, -0.3320]],

         [[-1.3640, -0.0372,  1.4103, -0.1755,  1.1940, -0.2636,  0.3186],
          [-1.9607,  0.4000,  1.9175, -1.7362,  0.4124,  1.0521, -0.6050],
          [-1.7676,  2.4814,  2.3001,  0.1844, -0.6243,  2.2404,  0.7621],
          [-1.7687, -0.2826,  0.0179, -0.3561,  0.3650,  0.8226,  0.6107]],

         [[-0.4571,  1.1342,  0.5213, -0.5604, -1.0119,  0.4766,  0.1784],
          [ 0.0692, -0.2373, -1.4442, -0.3252, -2.1491,  2.8538,  0.1160],
          [-2.1918,  0.3397,  1.0173, -0.2993, -2.6630,  0.7206,  0.1622],
          [-0.5263,  1.0427,  0.5387,